In [1]:
# FIRST PART OF PROGRAM CODE: USER-DEFINED FUNCTION FOR CLOCK
# create class to provide real-time day, date, and time
def get_time():
    # get current time in this format: Monday, 12/31/2023 23:59:59
    currentTime = datetime.now().strftime('%A, %m/%d/%Y %H:%M:%S')
    
    # return currentTime to function that requests it
    return currentTime

In [2]:
# SECOND PART OF PROGRAM CODE: DATA ANALYSIS (CONFIRMED WORKING)
# create a callable function that handles data query and putting collected data in dataframes
def data_query_and_dataframing():
    # debugging: print a message telling that this function is successfully called
    # print('data_query_and_dataframing() function called successfully')
    
    # create three variables to store server, database, and trusted device details, respectively
    server = 'LAPTOP-PLVL436R'
    database = 'isuzugencars'
    trusted = 'yes'
    
    # read contents of server.txt to grab its content
    # with open('server.txt', 'r') as file:
        # server = file.read()
    # database = 'isuzugencars'
    # username = 'sa'
    # password = 'gencars@dominga3'
    
    # use pyodbc.connect() function to connect to MS SQL Server specified above
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';TRUSTED_CONNECTION=' + trusted)
    # conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    
    # create a cursor for executing SQL commands
    c = conn.cursor()
    
    # modifiable query code for counter column which selects all records of jrhead not in jehead
    counterQuery = """SELECT jrno, plateno, csno, appointment
                      FROM dbo.tbl_jrhead
                      WHERE jrno NOT IN (SELECT jrno FROM dbo.tbl_jehead)
                      AND jc1 = 1
                      AND jrstatus = 0
                      AND jrdate >= DATEADD(day, -7, GETDATE())
                      ORDER BY jrdate DESC"""

    # modifiable query code for in progress column which selects all records of jrhead with jestatus = 0
    progressQuery = """SELECT jr.jrno, jr.plateno, jr.csno, jr.appointment
                      FROM dbo.tbl_jrhead jr
                      INNER JOIN (SELECT plateno, MAX(transdate) AS latest_date
                      FROM dbo.tbl_jehead GROUP BY plateno) je
                      ON jr.plateno = je.plateno
                      WHERE jr.jrno IN (SELECT jrno FROM dbo.tbl_jehead)
                      AND jr.jc1 = 1
                      AND jr.jrstatus = 0
                      AND jr.jrdate >= DATEADD(day, -7, GETDATE())
                      ORDER BY je.latest_date DESC"""

    # modifiable query code for completed column which selects all records of jrhead with jestatus = 1 and rono = jrno
    completedQuery = """SELECT jr.jrno, jr.plateno, jr.csno, jr.appointment
                      FROM dbo.tbl_jrhead jr
                      INNER JOIN (SELECT headuid, MAX(time2) AS latest_date
                      FROM dbo.tbl_jr_techlog GROUP BY headuid) jrtech
                      ON jr.uid = jrtech.headuid
                      WHERE jr.jrno IN (SELECT jrno FROM dbo.tbl_jehead)
                      AND jr.jrno IN (SELECT jrno FROM dbo.tbl_rohead WHERE jrno = rono)
                      AND jr.jc1 = 1
                      AND jr.jrstatus = 1
                      AND jr.jrdate >= DATEADD(day, -7, GETDATE())
                      ORDER BY jrtech.latest_date DESC"""
    
    # use the created cursor to execute and fetch data of the stored query codes of each column
    counterData = c.execute(counterQuery).fetchall()
    progressData = c.execute(progressQuery).fetchall()
    completedData = c.execute(completedQuery).fetchall()

    # use pd.dataframe() function to store fetched data of each column data in their respective dataframes
    counterDf = pd.DataFrame(counterData, columns = ['counterTuple'])
    progressDf = pd.DataFrame(progressData, columns = ['progressTuple'])
    completedDf = pd.DataFrame(completedData, columns = ['completedTuple'])

    # create lists to prepare columns in preparation for tuple splitting
    columnHeaders = ['jrno', 'plateno', 'csno', 'appointment']

    # use a for-loop and counterHeaders to split tuples stored in counterDf into their respective columns
    for n, col in enumerate(columnHeaders):
        counterDf[col] = counterDf['counterTuple'].apply(lambda location: location[n])

    # use a for-loop and progressHeaders to split tuples stored in progressDf into their respective columns
    for n, col in enumerate(columnHeaders):
        progressDf[col] = progressDf['progressTuple'].apply(lambda location: location[n])

    # use a for-loop and completedHeaders to split tuples stored in completedDf into their respective columns
    for n, col in enumerate(columnHeaders):
        completedDf[col] = completedDf['completedTuple'].apply(lambda location: location[n])

    # use df.drop() function to drop the columns of each column dataframes that still holds fetched tuples
    counterDf = counterDf.drop('counterTuple', axis = 1)
    progressDf = progressDf.drop('progressTuple', axis = 1)
    completedDf = completedDf.drop('completedTuple', axis = 1)
    
    # close the created cursor used to do queries
    c.close()

    # close the MS SQL Server 2014 connection
    conn.close()
    
    # return the three dataframes that will be used for preprocessing and plate preparation
    return counterDf, progressDf, completedDf

In [3]:
# THIRD PART OF PROGRAM CODE: USER-DEFINED FUNCTION FOR WITHIN COLUMN DEDUPLICATOR (CONFIRMED WORKING)
# create a function that will check and remove in-column duplicates of the passed columnDataframe
def within_column_deduplicator(columnData, columnName):
    # debugging: print a message telling that this function is successfully called
    # print('within_column_dedpulicator() function called successfully')
    
    # if the currently processed columnDf is not empty, proceed with the duplicate checking process
    if(not columnData.empty):
        # debugging: variable to keep track of dropped duplicates count
        # duplicatesNo = 0
        
        # for-loop that will loop through the unique content of the passed dataframe's 'plateno' column
        for plate in columnData['plateno'].unique():
            # create a new dataframe containing rows of detected duplicates of the current plate
            plateDuplicates = columnData[columnData['plateno'] == plate]

            # if the number of found duplicates stored in plateDuplicates is greater than 0...
            if len(plateDuplicates) > 0:
                # ...temporarily grab the jrno value of the current duplicate...
                highestJrno = plateDuplicates['jrno'].iloc[0]

                # ...then a for-loop will compare plateDuplicates unique element's jrno with highest_jrno
                for index, row in plateDuplicates.iterrows():    # .iterrows() function is used to iterate through the rows of plate_duplicates dataframe
                    # if the current row's jrno is less than the highestJrno...
                    if row['jrno'] < highestJrno:
                        # ...drop the current row to keep the unique element with the highest jrno
                        columnData = columnData.drop(index)
                        
                        # debugging: print the row that will be dropped, and add 1 to the current dropped duplicates counter
                        # print('Will drop ' + str(row))
                        # duplicatesNo = duplicatesNo + 1
        
        # debugging: print final counts of dropped duplicates
        # print('Number of removed ' + columnName + ' in-column duplicates: ' + str(duplicatesNo) + '\n')

    # return processed dataframe of a column, now free of duplicates within itself
    return columnData

In [4]:
# FOURTH PART OF PROGRAM CODE: USER-DEFINED FUNCTION FOR EXTERNAL COLUMNS DEDUPLICATOR (CONFIRMED WORKING)
# create a function that will check and remove external column duplicates of the passed columnDataframe
def external_columns_deduplicator(columnData, external1, external2, columnName, external1Name, external2Name):
    # debugging: print a message telling that this function is successfully called
    # print('external_columns_deduplicator() function called successfully')
    
    # if the currently processed columnDf is not empty, proceed with the duplicate checking process
    if(not columnData.empty):
        # if the passed external1 is not empty, proceed with the duplicate checking process of columnDf and external1
        if(not external1.empty):
            # debugging: variable to keep track of dropped duplicates count
            # duplicatesNo = 0
            
            # for-loop that will loop through the unique content of the passed dataframe's 'plateno' column
            for plate in columnData['plateno'].unique():
                # create two new dataframes, one for columnData and another for external1, containing rows of detected duplicates of the current plate
                plateDuplicates1 = columnData[columnData['plateno'] == plate]
                plateDuplicates2 = external1[external1['plateno'] == plate]
                
                # if the number of found duplicates stored in the two plateDuplicates variables are both greater than 0...
                if len(plateDuplicates1) > 0 and len(plateDuplicates2) > 0:
                    # ...use .max() function to grab the maximum jrno values of the duplicates in the two plate_duplicates dataframes, then grab store the higher value in highest_jrno...
                    highestJrno = max(plateDuplicates1['jrno'].max(), plateDuplicates2['jrno'].max())

                    # ...then a for-loop will compare plateDuplicates1 unique element's jrno with highestJrno
                    for index, row in plateDuplicates1.iterrows():
                        # if the current row's jrno is less than the highestJrno...
                        if row['jrno'] < highestJrno:
                            # ...drop the current row to keep the unique element with the highest jrno
                            columnData = columnData.drop(index)
                            
                            # debugging: print the row that will be dropped, and add 1 to the current dropped duplicates counter
                            # print('Will drop ' + str(row))
                            # duplicatesNo = duplicatesNo + 1
            
            # debugging: print final counts of dropped duplicates between the main column and external1 dataframes
            # print('Number of removed ' + columnName + ' to ' + external1Name + ' column duplicates: ' + str(duplicatesNo) + '\n')
        
        # if the passed external2 is not empty, proceed with the duplicate checking process of columnDf and external2
        if(not external2.empty):
            # debugging: variable to keep track of dropped duplicates count
            # duplicatesNo = 0
            
            # for-loop that will loop through the unique contents of the passed dataframe's 'plateno' column
            for plate in columnData['plateno'].unique():
                # create two new dataframes, one for columnData and another for external2, containing rows of detected duplicates of the current plate
                plateDuplicates1 = columnData[columnData['plateno'] == plate]
                plateDuplicates3 = external2[external2['plateno'] == plate]
                
                # if the number of found duplicates stored in the two plateDuplicates variables are both greater than 0...
                if len(plateDuplicates1) > 0 and len(plateDuplicates3) > 0:
                    # ...use .max() function to grab the maximum jrno values of the duplicates in the two plateDuplicates dataframes, then grab store the higher value in highest_jrno...
                    highestJrno = max(plateDuplicates1['jrno'].max(), plateDuplicates3['jrno'].max())

                    # ...then a for-loop will compare plateDuplicates1 unique element's jrno with highestJrno
                    for index, row in plateDuplicates1.iterrows():
                        # if the current row's jrno is less than the highestJrno...
                        if row['jrno'] < highestJrno:
                            # ...drop the current row to keep the unique element with the highest jrno
                            columnData = columnData.drop(index)
                            
                            # debugging: print the row that will be dropped, and add 1 to the current dropped duplicates counter
                            # print('Will drop ' + str(row))
                            # duplicatesNo = duplicatesNo + 1
            
            # debugging: print final counts of dropped duplicates between the main column and external2 dataframes
            # print('Number of removed ' + columnName + ' to ' + external2Name + ' column duplicates: ' + str(duplicatesNo) + '\n')
    
    # return processed dataframe of a column, now free of duplicates when compared with other columns
    return columnData

In [5]:
# FIFTH PART OF PROGRAM CODE: USER-DEFINED FUNCTION FOR PLATE AND COLOR PREPARATOR (CONFIRMED WORKING)
# create an automated plate and color preparing function for each plate number, which will take a columnDf data content
def plate_preparator(columnData):
    # debugging: print a message telling that this function is successfully called
    # print('plate_preparator() function called successfully')
    
    # create a variable that will store values of jrno as index
    indexColumn = columnData['jrno'].unique()

    # create a list to store biglist (aka list of lists of lists of plateno and its respective jrno)
    bigList = []
    
    # for-loop that will grab all plateno and appointment data where jrno equals current value
    for index in indexColumn:
        # use loc[] function to access current row matching the index value...
        # ...but only grab the stored values in plateno and appointment columns...
        # ...then use values.tolist() function to turn the grabbed values into a list 
        dataGrab = columnData.loc[columnData['jrno'] == index, ['plateno', 'csno', 'appointment']].values.tolist()
        
        # append datagrab to biglist
        bigList.append((dataGrab))
    
    # create a list to store generatorData (aka list of lists transformed using bigList's list of lists of lists)
    generatorData = []
    
    # for-loop that will transform biglist into smalllist
    for smallList in bigList:
        # append curent index 0 of smallList to generatorData
        generatorData.append(smallList[0])
    
    # return generatorData that will be used to generate colored plateno labels
    return generatorData

In [6]:
# SIXTH PART OF PROGRAM CODE: USER-DEFINED FUNCTION FOR PLATE UPDATER (CONFIRMED WORKING)
# BASICALLY THIS IS GET_TIME BUT FOR PLATE UPDATES, AND MUST RETURN PLATES OF EACH COLUMN
# create an automated destroy and recreate function for existing label widgets of plates
def plate_updater():
    # debugging: print a message telling that this function is successfully called
    # print('plate_updater() function called successfully')
    
    # call data_query_and_dataframing() function then unpack its returned tuple into their respective variables
    counterDf, progressDf, completedDf = data_query_and_dataframing()
    
    # now pass the columnDfs into the within_column_deduplicator() function to remove outdated duplicates within the respective columns
    counterDf = within_column_deduplicator(counterDf, 'Counter')
    progressDf = within_column_deduplicator(progressDf, 'In Progress')
    completedDf = within_column_deduplicator(completedDf, 'Completed')
    
    # now pass the columnDfs into the external_columns_deduplicator() function to remove outdated duplicates found in other columns that is not the currently processed column
    counterDf = external_columns_deduplicator(counterDf, progressDf, completedDf, 'Counter', 'In Progress', 'Completed')
    progressDf = external_columns_deduplicator(progressDf, completedDf, counterDf, 'In Progress', 'Completed', 'Counter')
    completedDf = external_columns_deduplicator(completedDf, counterDf, progressDf, 'Completed', 'In Progress', 'Completed')

    # call variables outside of function and set them as global
    global counterPlates, progressPlates, completedPlates
    
    # put all plateno data of counterDf, progressDf, and completedDf dataframe into their respective lists
    counterPlates = plate_preparator(counterDf)
    progressPlates = plate_preparator(progressDf)
    completedPlates = plate_preparator(completedDf)

    # turn the three final, prepared results into a dictionary variable called allPlates
    allPlates = {'counter': counterPlates, 'progress': progressPlates, 'completed': completedPlates}
    
    # return allPlates that will be used to generate colored plateno labels by JavaScript
    return allPlates

In [7]:
# SEVENTH PART OF PROGRAM CODE: INITIALIZATION
# import dependencies
from datetime import datetime
from flask import Flask, render_template, jsonify
import logging
import pandas as pd
import pyodbc

# configure logging to only show error level messages
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

# create a Flask web application instance
app = Flask(__name__)

# define path to function that will return current time as JSON object
@app.route('/time', methods = ['GET'])

# create user-defined function for current date grabber
def time_grab():
    # call get_time() function and return its result in time variable
    time = get_time()
    
    # return time to base template as json file using jsonify
    return jsonify(result = time)

# define path to function that will return current allPlates as JSON object
@app.route('/plates', methods = ['GET'])

# create user-defined function for current allPlates grabber
def plates_grab():
    # call plate_updater() function and return its result in plates variable
    #allPlates = {"completed":[["im completed", "hotdog", "yes"], ["youre completed", "cheesedog", "no"]],"counter":["im counter"],"progress":["im in progress", "youre in progress"]}
    allPlates = plate_updater()
    
    # return plates to base template as json file using jsonify
    return jsonify(allPlates)

# define path to activate function below
@app.route('/')

# create user-defined function for service board to return what to display at user
def service_board():
    # use render_template() function to grab HTML template and return to display said template
    return render_template('base.html')

# if-statement to run Flask web application instance
if __name__ == '__main__':
    print('Service Board Link: http://127.0.0.1:5000/')
    app.run()

Service Board Link: http://127.0.0.1:5000/
 * Serving Flask app '__main__'
 * Debug mode: off
